In [2]:
import pandas as pd
import numpy as np
import re

## Common Data Cleaning Tasks

* Handling missing data
* 

In [2]:
df = pd.DataFrame([[1., 6.5, 3.0, 0.5], [1., np.nan, 3, 1.5],
                     [np.nan, np.nan, np.nan, np.nan], [np.nan, 6.5, np.nan, 3.5],
                     [np.nan, 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [3]:
df

,A,B,C,D
0,1.0,6.5,3.0,0.5
1,1.0,NaN,3.0,1.5
2,NaN,NaN,NaN,NaN
3,NaN,6.5,NaN,3.5
4,NaN,6.5,3.0,3.5


## Missing data

Data that is to be used in a machine or statistical learning model needs to be absent of missing values.  Below are some ways to handle missing values in pandas.  The "correct" way to treat missing values will be context specific.  Analyses might compare results from how missing values were handled differently.  Often, missing values are filled using `Scikit-Learn` so the process can be built into a pipeline. 

1. **Detecting Missing Values**:
    - `isna()` or `isnull()`: Returns a DataFrame or Series indicating which values are missing.
    - `notna()` or `notnull()`: Opposite of `isna()`, indicates which values are not missing.
  
2. **Removing Missing Values**:
    - `dropna()`: 
        - Remove missing values.
        - Can specify `axis` (rows or columns).
        - Can use `how` argument to specify 'any' (default) or 'all' to determine when rows/columns are dropped.
        - Can use `subset` argument to specify to only consider certain  columns or rows

3. **Filling Missing Values**:
    - `fillna()`:
        - Fill missing values with a specific value.
        - Use `method` argument to specify method ('ffill' for forward fill, 'bfill' for backward fill).
        - Can fill using the mean, median, or mode.
        - Can also use interpolation methods.

4. **Replace Missing Values**:
    - `replace()`: 
        - Replace specified values with another value (can be used for values other than NaN too).

5. **Using an Indicator**:
    - Add a new column to indicate where values are missing: 
        ```python
        df['missing_col_indicator'] = df['col'].isnull()
        ```

6. **Group-wise Handling**:
    - Use `groupby` along with `fillna()` to fill missing values based on some group properties:
        ```python
        df['col'] = df.groupby('group_col')['col'].transform(lambda x: x.fillna(x.mean()))
        ```

7. **Forward and Backward Filling**:
    - `ffill()` or `pad()`: Fill values forward.
    - `bfill()` or `backfill()`: Fill values backward.

8. **Limiting Fill or Drop Actions**:
    - Using the `limit` parameter in methods like `dropna()` and `fillna()` to restrict the **number** of rows/columns affected.
    - Using the `subset` argument in methods like `dropna()` and `fillna()` to specify the actual row/columns affected.



In [4]:
df.isna().sum() #number of missing values in each column

A    3
B    2
C    2
D    1
dtype: int64

In [5]:
df.dropna(subset = ['D']) #drops the rows in D that have NaN

,A,B,C,D
0,1.0,6.5,3.0,0.5
1,1.0,NaN,3.0,1.5
3,NaN,6.5,NaN,3.5
4,NaN,6.5,3.0,3.5


In [7]:
df.dropna(subset = ['D', 'C'], how = 'any') #drop values that are missing in both C or D

,A,B,C,D
0,1.0,6.5,3.0,0.5
1,1.0,NaN,3.0,1.5
4,NaN,6.5,3.0,3.5


In [8]:
df.dropna(subset = ['D', 'C'], how = 'all') #only drop values that are missing in both C and D

,A,B,C,D
0,1.0,6.5,3.0,0.5
1,1.0,NaN,3.0,1.5
3,NaN,6.5,NaN,3.5
4,NaN,6.5,3.0,3.5


In [10]:
df.fillna(value = df.mean()) #fill with mean of column

,A,B,C,D
0,1.0,6.5,3.0,0.50
1,1.0,6.5,3.0,1.50
2,1.0,6.5,3.0,2.25
3,1.0,6.5,3.0,3.50
4,1.0,6.5,3.0,3.50


---

## Duplicates

Handling duplicate data is important, especially in data preprocessing, to ensure the accuracy of subsequent analyses. Here's a bullet list of possible ways to check for and handle duplicate data in pandas:

1. **Checking for Duplicates**:
    - `duplicated()`: Returns a Boolean series that indicates whether each row is a duplicate or not.

        ```python
        df = pd.DataFrame({'A': [1, 2, 2, 3, 3], 'B': [1, 1, 2, 2, 3]})
        print(df.duplicated())
        ```

2. **Counting Duplicates**:
    - Chain the `sum()` function after `duplicated()` to get the total count of duplicate rows.

        ```python
        print(df.duplicated().sum())
        ```

3. **Viewing Duplicates**:
    - Use boolean indexing to filter and see the actual duplicate rows.

        ```python
        print(df[df.duplicated()])
        ```

4. **Removing Duplicates**:
    - `drop_duplicates()`: Removes the duplicate rows from the DataFrame.

        ```python
        df_no_duplicates = df.drop_duplicates()
        ```

5. **Considering Specific Columns**:
    - `drop_duplicates(subset=column_name)`: Check for duplicates based on specific columns.

        ```python
        df_no_dup_in_A = df.drop_duplicates(subset='A')
        ```

6. **Keeping Specific Duplicates**:
    - The `keep` parameter in `drop_duplicates()`:
        - `keep='first'` (default): Keeps the first occurrence of the duplicate.
        - `keep='last'`: Keeps the last occurrence of the duplicate.
        - `keep=False`: Removes all occurrences of the duplicates.

        ```python
        df_removed_all = df.drop_duplicates(keep=False)
        ```

7. **Checking Duplicates in Columns**:
    - Sometimes you might want to check duplicates within a column.

        ```python
        print(df['A'].duplicated().sum())
        ```

8. **Removing Duplicates in Columns**:
    - To drop duplicate values in a specific column, you can use `drop_duplicates` with the `subset` argument.

        ```python
        df_no_dup_column = df.drop_duplicates(subset='A', keep='first')
        ```

9. **Resetting Index**:
    - After dropping duplicates, you might have gaps in your index. You can reset the index using `reset_index(drop=True)`.

        ```python
        df_reset = df_no_duplicates.reset_index(drop=True)
        ```

10. **Considering Data Order**:
    - When handling duplicates, the order of data can matter. Ensure you've sorted the data (if necessary) before addressing duplicates.


In [11]:
df.drop_duplicates(subset=["B","D"])

,A,B,C,D
0,1.0,6.5,3.0,0.5
1,1.0,NaN,3.0,1.5
2,NaN,NaN,NaN,NaN
3,NaN,6.5,NaN,3.5


---

## Outliers

Handling outliers is essential as they can distort results and lead to incorrect conclusions. However, it is important to remember that not all outliers are bad or erroneous. Like many data cleaning steps, the appropriate way to handle outliers can depend on the problem.

Here are a few possible ways to check for and handle outliers in pandas:



1. **Visualization**:
    - Using plots can help in visually identifying outliers.
    - `boxplot()`: A box plot can reveal outliers as points beyond the "whiskers" of the box plot.

        ```python
        import matplotlib.pyplot as plt
        df = pd.DataFrame({'Values': [1, 2, 3, 50]})
        df.boxplot(column='Values')
        plt.show()
        ```

2. **Descriptive Statistics**:
    - `describe()`: Gives a summary of statistics, where the minimum and maximum values can provide insights about potential outliers.
    
        ```python
        print(df.describe())
        ```

3. **Z-Score**:
    - Measure the number of standard deviations a data point is from the mean.
    - Data points with a z-score greater than 3 or less than -3 are typically considered outliers.
        ```python
        from scipy.stats import zscore
        abs_z_scores = np.abs(zscore(df['Values']))
        outliers = df[abs_z_scores > 3]
        print(outliers)
        ```

4. **IQR (Interquartile Range)**:
    - IQR is the range between the first quartile (25th percentile) and the third quartile (75th percentile).
    - Data points outside 1.5 times the IQR below the first quartile or above the third quartile might be considered outliers.

        ```python
        Q1 = df['Values'].quantile(0.25)
        Q3 = df['Values'].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[((df['Values'] < (Q1 - 1.5 * IQR)) | (df['Values'] > (Q3 + 1.5 * IQR)))]
        print(outliers)
        ```

5. **Removing Outliers**:
    - After identifying outliers, you can drop them using boolean indexing.

        ```python
        filtered_df = df[(abs_z_scores < 3)]
        ```

6. **Capping Outliers**:
    - Instead of removing outliers, you can limit their value to a specific range (capping).

        ```python
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df['Values'] = np.where(df['Values'] > upper_bound, upper_bound, df['Values'])
        df['Values'] = np.where(df['Values'] < lower_bound, lower_bound, df['Values'])
        ```

7. **Transforming Data**:
    - Outliers can sometimes be managed by transforming data to fit into a more regular shape.
    - Common transformations include square root, log, and box-cox transformations.

        ```python
        df['Log_Values'] = np.log(df['Values'] + 1)
        ```


In [12]:
np.random.seed(100)
data = pd.DataFrame(np.random.standard_normal((10, 4)))
data = data*3

In [13]:
data

,0,1,2,3
0,-5.249296,1.028041,3.459107,-0.757308
1,2.943962,1.542657,0.663539,-3.210130
2,-0.568487,0.765004,-1.374081,1.305490
3,-1.750785,2.450541,2.018162,-0.313233
4,-1.593841,3.089198,-1.314407,-3.354955
5,4.856945,4.624816,-0.755637,-2.527307
6,0.553556,2.811247,2.193001,4.084668
7,-0.978714,0.167028,0.667199,-4.329651
8,-2.269057,2.449362,2.251334,-1.367841
9,3.568867,-5.071850,-4.069197,-3.697304


In [14]:
data[data[0].abs() > 4]

,0,1,2,3
0,-5.249296,1.028041,3.459107,-0.757308
5,4.856945,4.624816,-0.755637,-2.527307


In [ ]:
data[(data.abs() > 4).any(axis=1)]

In [ ]:
# replace outliers with .mask()

`.mask` is an application of the if-then idiom.  If "condition" is True, replace with "value".  There are also `inplace` and `axis` arguments.

`df.mask(condition, value)`

In [ ]:
data.quantile(.9)

In [ ]:
data_fixed = data.mask(data>4, data.quantile(.9), axis=1)

In [ ]:
data_fixed = data_fixed.mask(data< -4, data.quantile(.1), axis=1)

In [ ]:
data.mask(data.abs()>4, np.nan)

---

## Discretizing

Another way to handle outliers, is to discretize or bin a continuous/numeric variable.  Other reasons for binning include (not an exhaustive list):
- Data simplification
- Visualization
- Dealing with noise and variability
- Improving model performance
- Handling sparse data areas
- Ensuring privacy
- Etc. 

In [15]:
s = data.iloc[:,0]

In [16]:
s

0   -5.249296
1    2.943962
2   -0.568487
3   -1.750785
4   -1.593841
5    4.856945
6    0.553556
7   -0.978714
8   -2.269057
9    3.568867
Name: 0, dtype: float64

In [17]:
bins = [-10,-2,-1,1,2,10]

In [18]:
pd.cut(s, bins)

0    (-10, -2]
1      (2, 10]
2      (-1, 1]
3     (-2, -1]
4     (-2, -1]
5      (2, 10]
6      (-1, 1]
7      (-1, 1]
8    (-10, -2]
9      (2, 10]
Name: 0, dtype: category
Categories (5, interval[int64, right]): [(-10, -2] < (-2, -1] < (-1, 1] < (1, 2] < (2, 10]]

In [19]:
pd.cut(s,bins).value_counts()

0
(-1, 1]      3
(2, 10]      3
(-10, -2]    2
(-2, -1]     2
(1, 2]       0
Name: count, dtype: int64

In [20]:
pd.cut(s,bins).value_counts()

0
(-1, 1]      3
(2, 10]      3
(-10, -2]    2
(-2, -1]     2
(1, 2]       0
Name: count, dtype: int64

In [21]:
pd.cut(s,bins,labels=['very low','low','average','high','very high'])

0     very low
1    very high
2      average
3          low
4          low
5    very high
6      average
7      average
8     very low
9    very high
Name: 0, dtype: category
Categories (5, object): ['very low' < 'low' < 'average' < 'high' < 'very high']

---

## Dummy Variables

Creating dummy variables is also known as One-hot encoding.  It is a possible method for converting categorical variables into numbers that can be fed into a machine or statistical learning model. 


1. **Binary Categorical Data**:
    - For binary categorical data (e.g., "Yes" or "No"), you can represent it with one dummy variable (1 for "Yes" and 0 for "No").

2. **Multi-category Categorical Data**:
    - For a categorical variable with *n* distinct categories, you can represent it using *n-1* or *n* dummy variables, depending on the approach.
    - Why *n-1* or *n*? Using *n-1* dummy variables avoids the "dummy variable trap" or multicollinearity, where one variable can be predicted from the others. However, in some situations, like with certain machine learning algorithms or when clarity is needed, all *n* dummy variables might be used.

### Example:

Let's take an example where a feature "Color" in a dataset has three categories: "Red", "Green", and "Blue".

Using dummy variables, this can be represented as:

| Color | Red | Green | Blue |
|-------|-----|-------|------|
| Red   | 1   | 0     | 0    |
| Green | 0   | 1     | 0    |
| Blue  | 0   | 0     | 1    |

If you choose to use *n-1* dummy variables to avoid multicollinearity, you might drop one of the columns. If you drop the "Blue" column, the absence of "Red" and "Green" (i.e., both being 0) would imply "Blue".

### Key Points:

- **Advantages**:
    - Makes categorical data usable in models that require numerical input variables.
    - Helps in revealing more granular patterns in the relationship between features and response variables.
  
- **Disadvantages**:
    - Can lead to a substantial increase in dataset dimensions, especially with high-cardinality categorical variables.
    - Can introduce multicollinearity if not handled correctly.

- **Alternatives**:
    - While dummy variables are a popular way to handle categorical data, there are other methods like ordinal encoding, binary encoding, and embedding layers (for deep learning models).

In `pandas`, the `pd.get_dummies()` function can be used to easily convert categorical variables into dummy/indicator variables.

In [22]:
df2 = pd.DataFrame([['apple', 6.5, 3.0, 0.5], ['apple', np.nan, np.nan, 1.5],
                     ['banana', np.nan, np.nan, np.nan], ['orange', 6.5, 3., 3.5],
                     ['orange', 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [ ]:
df2

In [23]:
pd.get_dummies(df2)

,B,C,D,A_apple,A_banana,A_orange
0,6.5,3.0,0.5,True,False,False
1,NaN,NaN,1.5,True,False,False
2,NaN,NaN,NaN,False,True,False
3,6.5,3.0,3.5,False,False,True
4,6.5,3.0,3.5,False,False,True


---
## Combining categories or other encodings

Instead of just using dummy variables, sometimes we might want to use an ordinal or other type of encoding.  One way to do this would be to create a mapping dictionary.  

```
mapping_dict = {old_value:new_value, 
another_old:another_new}

## apply
df['category'].map(mapping_dict)
```

In [27]:
mapping = {"apple": "Group 1",
           "banana": "Group 2",
           "orange": "Group 3"}
df2["new"]=df2["A"].map(mapping)
df2

,A,B,C,D,new
0,apple,6.5,3.0,0.5,Group 1
1,apple,NaN,NaN,1.5,Group 1
2,banana,NaN,NaN,NaN,Group 2
3,orange,6.5,3.0,3.5,Group 3
4,orange,6.5,3.0,3.5,Group 3


---

## Changing data types (casting)

Casting is the process of converting one data type into another.  Changing strings to numbers and numbers to strings is a common occurrence when cleaning data. Some pandas functions and methods used for casting include:


1. **astype()**:
    - The primary method to cast a pandas object to a specified data type.
    ```python
    df['column_name'] = df['column_name'].astype('new_data_type')
    ```

2. **to_numeric()**:
    - Convert a Series or a single column of a DataFrame to a numeric data type (either integer or float).
    - Useful for converting columns that might have been read as strings or objects due to the presence of non-numeric values.
    ```python
    df['column_name'] = pd.to_numeric(df['column_name'], errors='coerce')
    ```

3. **to_datetime()**:
    - Convert a Series or column to a datetime data type.
    ```python
    df['date_column'] = pd.to_datetime(df['date_column'])
    ```

4. **to_timedelta()**:
    - Convert a Series or column to a timedelta data type.
    ```python
    df['duration_column'] = pd.to_timedelta(df['duration_column'])
    ```

5. **pd.Categorical()**:
    - Convert a Series or column to a categorical data type.
    ```python
    df['category_column'] = pd.Categorical(df['category_column'])
    ```

When casting variables, it's essential to be aware of potential data loss or errors. For instance, casting a floating-point column to an integer will truncate the decimal parts, and trying to convert non-numeric strings to integers or floats can raise errors. Always check the data after casting to ensure that the conversion has been done correctly and as expected.

In [ ]:
df3 = pd.DataFrame([['apple', '6.5', 3.0, 0.5], ['apple', np.nan, np.nan, 1.5],
                     ['banana', np.nan, np.nan, np.nan], ['orange', 6.5, 3., 3.5],
                     ['orange', 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [ ]:
df3

In [ ]:
df3.info()

In [ ]:
df3['B'] = df3['B'].astype(float)

In [ ]:
df3.info()

---

## Unpacking

Many times, a single column will contain multiple pieces of information.  Learning how to extract or unpack this information is a common data cleaning task. 

Python's built in string methods and regular expressions are very helpful in these instances.  (See Python for Data Analysis Section 7.4)

### Helpful built in string methods

- `join`: Use string a delimiter for concatenating a sequence of other strings
- `strip`: Trim whitespace - Can also remove characters in the `chars` argument
- `split`: Split into a list of substrings based white space or `sep` argument
- `lower`: Converts to lowercase
- `replace`: Replace substring with another substring

### Other functions

Other functions, attributes, and methods that are commonly used in unpacking tasks include

- `.apply`: Use to pally a function along the axis (either rows or columns) of a DataFrame or on the elements of a Series.
- `lambda` functions
- Accessor objects allow you to perform operations on a Series without the need of loops    
    - `.str`: An accessor object used for vectorized string operations on a pandas Series
    - `.dt`: An accessor object used for vectorized date operations on a pandas datetime Series
    - *Further explanation:*   String and regular expression methods/functions  only work on `string` objects.  Date function/methods only work on `datetime` objects.  When dealing with pandas, the *value* inside a Series might be a `string` or `datetime`, but the Series itself is **not** a `string`.  Calling `.str` or .`dt` off of a Series allows `string` and `datetime` functions to be used on the Series.
    ```python
    s = pd.Series(['apple', 'banana', 'cherry'])    
    s.str[:3]
    s.str.startswith('a')

    dates = pd.Series(pd.date_range("2022-01-01", periods=3, freq="Y"))
    dates.dt.year
    ```



---
# Tidy Data
---

## Melting: wide to long

In [3]:
pew = pd.read_csv('Data/pew-raw.csv')
pew.head(2)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70


In [4]:
pew

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


In [5]:
pd.melt(pew, id_vars=['religion',' <$10k'])

,religion,<$10k,variable,value
0,Agnostic,27,$10-20k,34
1,Atheist,12,$10-20k,27
2,Buddhist,27,$10-20k,21
3,Catholic,418,$10-20k,617
4,Dont know/refused,15,$10-20k,14
5,Evangelical Prot,575,$10-20k,869
6,Hindu,1,$10-20k,9
7,Historically Black Prot,228,$10-20k,244
8,Jehovahs Witness,20,$10-20k,27
9,Jewish,19,$10-20k,19


In [6]:
varnames = [' <$10k', ' $10-20k', '$20-30k', '$30-40k', ' $40-50k',
       '$50-75k']

In [7]:
pd.melt(pew, id_vars='religion',value_vars=varnames)

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Dont know/refused,<$10k,15
5,Evangelical Prot,<$10k,575
6,Hindu,<$10k,1
7,Historically Black Prot,<$10k,228
8,Jehovahs Witness,<$10k,20
9,Jewish,<$10k,19


In [ ]:
bb.columns

In [8]:
bb = pd.read_csv('Data/billboard.csv', encoding='latin1')
bb

,year,artist,track,time,genre,date,peak-date,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,8/5/00,8/5/00,98,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,2000,"Smith, Will",Freakin' It,3:58,Rap,2/12/00,2/12/00,99,99.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,2000,Zombie Nation,Kernkraft 400,3:30,Rock,9/2/00,9/2/00,99,99.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,2000,"Eastsidaz, The",Got Beef,3:58,Rap,7/1/00,7/1/00,99,99.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.melt(bb, id_vars=['year','artist'],value_vars=bb.columns[7:])

,year,artist,variable,value
0,2000,Destiny's Child,x1st.week,78.0
1,2000,Santana,x1st.week,15.0
2,2000,Savage Garden,x1st.week,71.0
3,2000,Madonna,x1st.week,41.0
4,2000,"Aguilera, Christina",x1st.week,57.0
...,...,...,...,...
24087,2000,Ghostface Killah,x76th.week,NaN
24088,2000,"Smith, Will",x76th.week,NaN
24089,2000,Zombie Nation,x76th.week,NaN
24090,2000,"Eastsidaz, The",x76th.week,NaN


In [10]:
pd.melt(bb, id_vars=bb.columns[0:7], var_name='week', value_name='rank')

,year,artist,track,time,genre,date,peak-date,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,x1st.week,57.0
...,...,...,...,...,...,...,...,...,...
24087,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,8/5/00,8/5/00,x76th.week,NaN
24088,2000,"Smith, Will",Freakin' It,3:58,Rap,2/12/00,2/12/00,x76th.week,NaN
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,9/2/00,9/2/00,x76th.week,NaN
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,7/1/00,7/1/00,x76th.week,NaN


## Pivoting:  long to wide

In [11]:
st = pd.read_csv('Data/stocks.csv')

In [12]:
st

,date,symbol,open,high,low,close,volume
0,2019-03-01,AMZN,1655.13,1674.26,1651.00,1671.73,4974877
1,2019-03-04,AMZN,1685.00,1709.43,1674.36,1696.17,6167358
2,2019-03-05,AMZN,1702.95,1707.80,1689.01,1692.43,3681522
3,2019-03-06,AMZN,1695.97,1697.75,1668.28,1668.95,3996001
4,2019-03-07,AMZN,1667.37,1669.75,1620.51,1625.95,4957017
5,2019-03-01,AAPL,174.28,175.15,172.89,174.97,25886167
6,2019-03-04,AAPL,175.69,177.75,173.97,175.85,27436203
7,2019-03-05,AAPL,175.94,176.00,174.54,175.53,19737419
8,2019-03-06,AAPL,174.67,175.49,173.94,174.52,20810384
9,2019-03-07,AAPL,173.87,174.44,172.02,172.50,24796374


In [14]:
temp = st.pivot(index = "date", columns ='symbol', values = ['open', 'close'])

In [19]:
temp

open                     close                  
symbol        AAPL     AMZN     GOOG    AAPL     AMZN     GOOG
date                                                          
2019-03-01  174.28  1655.13  1124.90  174.97  1671.73  1140.99
2019-03-04  175.69  1685.00  1146.99  175.85  1696.17  1147.80
2019-03-05  175.94  1702.95  1150.06  175.53  1692.43  1162.03
2019-03-06  174.67  1695.97  1162.49  174.52  1668.95  1157.86
2019-03-07  173.87  1667.37  1155.72  172.50  1625.95  1143.30

In [15]:
temp.index

Index(['2019-03-01', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07'], dtype='object', name='date')

In [16]:
temp.columns

MultiIndex([( 'open', 'AAPL'),
            ( 'open', 'AMZN'),
            ( 'open', 'GOOG'),
            ('close', 'AAPL'),
            ('close', 'AMZN'),
            ('close', 'GOOG')],
           names=[None, 'symbol'])

In [ ]:
wide_table = pd.pivot(st, index='date', columns='symbol')

In [ ]:
wide_table.columns

In [ ]:
wide_table.index

In [ ]:
wide_table.reset_index()

In [ ]:
st_new = pd.pivot(st, index='date', columns='symbol', values=['open'])

In [ ]:
st_new.reset_index()

In [ ]:
st_new.info()

In [ ]:
st_new.reset_index()

In [ ]:
country = pd.read_csv('Data/country_data2.csv')

In [ ]:
country.head(10)

In [ ]:
pd.pivot(country, index=['Country','Region'], columns='Variable', values='Value')

In [ ]:
a = pd.pivot_table(country,index=['Country','Region'], 
         columns='Variable', values='Value').reset_index()

In [ ]:
a.index

## Times

In [ ]:
from datetime import datetime

In [ ]:
a = datetime.now()
# code
b = datetime.now()

In [ ]:
b-a

In [ ]:
a.hour

In [ ]:
## Datetime in pandas

In [ ]:
aqi = pd.read_csv('Data/data.csv', index_col='date', parse_dates=True)

In [ ]:
aqi.head()

In [ ]:
aqi.index

In [ ]:
# get all 2021 
aqi.loc['2021']

In [ ]:
# get all dates Apr 15, 2021 and May 15, 2021
aqi.loc['2021-04-15':'2021-05-15']

In [ ]:
aqi['Utah'].plot()

In [ ]:
# create a day of year variable
aqi.index.day_of_year